# Decision Trees

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import classification_report

%matplotlib inline 

## 1. Importando dataset

In [4]:
dataset = pd.read_csv('eda/dataset.csv')
dataset.head()

,age,default,balance,housing,loan,day,month,campaign,pdays,previous,...,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown
0,58,0,2143,1,0,5,5,1,-1,0,...,0,1,0,0,0,1,0,0,0,1
1,44,0,29,1,0,5,5,1,-1,0,...,0,0,1,0,1,0,0,0,0,1
2,33,0,2,1,1,5,5,1,-1,0,...,0,1,0,0,1,0,0,0,0,1
3,47,0,1506,1,0,5,5,1,-1,0,...,0,1,0,0,0,0,1,0,0,1
4,33,0,1,0,0,5,5,1,-1,0,...,0,0,1,0,0,0,1,0,0,1


## 2. Preparando dataset

1. Remover atributo target do dataset X (usando drop) 
2. Obter coluna target do dataset
3. Separar dados de treino de dados de teste (nesse caso 77% para treino e 33% para teste)

In [5]:
X = dataset.drop('aceitaram',axis=1)
y = dataset['aceitaram']
X_train, X_test, y_train, y_test = train_test_split(X.values,y,test_size=0.33,random_state=42)

## 3. Criando modelo
1. Definir o método de aprendizagem (árvore de decisões)
2. Definir hiperparâmetros (inicialmente default)
3. "Ajustar" modelo (.fit) com os dados de treino

In [6]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

## 4. Testando modelo
1. Com o modelo criado, a partir dos dados de treino, pode-se testar o modelo com os dados de teste
2. Aplicando os dados de teste é possível verificar o desempenho do modelo
    * **precision**: A precisão é a capacidade de um classificador de não rotular uma instância positiva que seja realmente negativa."Para todas as instâncias classificadas como positivas, qual porcentagem estava correta?"  **precision = tp/(tp+fp)**
    * **recall**: É a capacidade de um classificador para encontrar todas as instâncias positivas. Para cada classe, é definida como a proporção de verdadeiros positivos para a soma de verdadeiros positivos e falsos negativos. "Para todas as instâncias que foram realmente positivas, qual porcentagem foi classificada corretamente?" **recall = tp/(tp+fn)**
    * **f1-score**: É uma média harmônica ponderada de _precision_ e _recall_ de tal forma que o melhor escore é 1,0 e o pior é 0,0. De um modo geral, as pontuações F1 são inferiores às medidas de precisão.
    * **support**:Suporte é o número de ocorrências reais da classe no conjunto de dados especificado. 
3. Verificando desempenho:
    * **precision**: Para a classe de obervações sim, o classificador obteve uma precisão de 91%. Para a classe de pessoas que responderam não, o classificador obteve uma precisão de 28%. O total de observações de teste que disseram não está em torno de 88% enquanto os que aceitaram, disseram sim, é de apenas 12%. O Resultado da precisão do modelo, sem nenhuma variação de parâmetros, ou seja, com os parâmetros default do modelo é de 83%.
    * **recall**: De todas as observações que tiveram predição dita como "Aceita" 32% foram mesmo aceitas, da mesma forma, das observações que tiverão predição como "Recusada" 89% foram mesmo recusadas. Resultando em 82% do total realmente era o que foi predita pelo modelo.

In [7]:
y_predict = clf.predict(X_test)
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Predicted Not Accept', 'Predicted Accept'],
    index=['True Not Accept', 'True Accept']
)

,Predicted Not Accept,Predicted Accept
True Not Accept,11719,1415
True Accept,1225,561


In [8]:
print(classification_report(y_test, y_predict,target_names=['Recusaram','Aceitaram']))
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_predict))

              precision    recall  f1-score   support

   Recusaram       0.91      0.89      0.90     13134
   Aceitaram       0.28      0.31      0.30      1786

   micro avg       0.82      0.82      0.82     14920
   macro avg       0.59      0.60      0.60     14920
weighted avg       0.83      0.82      0.83     14920

0.8230563002680965


## 5. Exibindo Árvore de Decisões
1. Exibir árvore a partir de graphviz
** É necessário a instalação do pacote graphviz **
> sudo apt-get install graphviz

In [9]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=X.columns.values.tolist(),  
                         class_names=['sim', 'não'],  
                         filled=True, rounded=True,  
                         special_characters=True)

graph = graphviz.Source(dot_data)  

In [10]:
# graph

## 6. Variando hiperparâmetros
1. Variação de parâmetros:
    * **criterion**: Pode ser "gini", para impurezas Gini, e "entropy", para ganho de informação.
    * **min_samples_leaf**: Indica o mínimo de objetos requeridos em um nó folha.
    * **min_samples_split**: Indica o número mínimo de objetos necessários para dividir um nó interno.
    * **max_depth**: Controla a profundidade máxima da árvore, se "None" aumentará a árvore até que todas as folhas fiquem puras ou até que todas as folhas contenham menos de min_samples_split.

In [11]:
clf = tree.DecisionTreeClassifier(criterion='gini',
                                  min_samples_leaf=5,
                                  min_samples_split=5,
                                  max_depth=None)
clf = clf.fit(X_train, y_train)

## 7. Verificando desempenho com hiperparâmetros modificados

In [12]:

y_predict = clf.predict(X_test)
pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Predicted Not Accept', 'Predicted Accept'],
    index=['True Not Accept', 'True Accept']
)

,Predicted Not Accept,Predicted Accept
True Not Accept,12362,772
True Accept,1301,485


In [13]:
print(classification_report(y_test, y_predict,target_names=['Recusaram','Aceitaram']))

              precision    recall  f1-score   support

   Recusaram       0.90      0.94      0.92     13134
   Aceitaram       0.39      0.27      0.32      1786

   micro avg       0.86      0.86      0.86     14920
   macro avg       0.65      0.61      0.62     14920
weighted avg       0.84      0.86      0.85     14920



In [14]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=X.columns.values.tolist(),  
                         class_names=['sim', 'não'],  
                         filled=True, rounded=True,  
                         special_characters=True)

graph = graphviz.Source(dot_data) 

In [15]:
# graph

In [18]:
report_precision = []

for index in range(1, 34):
    clf = tree.DecisionTreeClassifier(criterion='gini',
                                  min_samples_leaf=5,
                                  min_samples_split=5,
                                  max_depth=index)
    clf = clf.fit(X_train, y_train) 
    report_precision.append(classification_report(y_test, y_predict, output_dict=True, target_names=['Recusaram','Aceitaram']))

In [22]:
report_precision[0]['macro avg']

{'f1-score': 0.6207024856778188,
 'precision': 0.6453093154802403,
 'recall': 0.6063889043780101,
 'support': 14920}